In [1]:
!python -V

Python 3.11.12


In [2]:
!git clone https://github.com/Rudrabha/Wav2Lip.git

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 406, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 406 (delta 0), reused 0 (delta 0), pack-reused 403 (from 1)
Receiving objects: 100% (406/406), 546.35 KiB | 2.95 MiB/s, done.
Resolving deltas: 100% (225/225), done.


In [3]:
!rm -rf /content/sample_data

In [4]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [5]:
!wget https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth

--2025-04-21 10:51:59--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘s3fd-619a316812.pth’

s3fd-619a316812.pth 100%[===================>]  85.68M  24.7MB/s    in 3.7s    

2025-04-21 10:52:04 (23.2 MB/s) - ‘s3fd-619a316812.pth’ saved [89843225/89843225]



In [6]:
!mkdir -p /content/Wav2Lip/face_detection/detection/sfd/

In [7]:
!mv /content/s3fd-619a316812.pth /content/Wav2Lip/face_detection/detection/sfd/s3fd.pth

In [8]:
import gdown
gdown.download("https://drive.google.com/uc?id=15G3U08c8xsCkOqQxE38Z2XXDnPcOptNk", quiet=False)
gdown.download("https://drive.google.com/uc?id=15MzTPHCbD4FGDh1bzKqgdgq2GZqgsNvD", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=15G3U08c8xsCkOqQxE38Z2XXDnPcOptNk
From (redirected): https://drive.google.com/uc?id=15G3U08c8xsCkOqQxE38Z2XXDnPcOptNk&confirm=t&uuid=8ffc3966-6790-4b3f-b61b-8f7e6eabc83d
To: /content/Wav2Lip-SD-GAN.pt
100%|██████████| 146M/146M [00:03<00:00, 43.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=15MzTPHCbD4FGDh1bzKqgdgq2GZqgsNvD
From (redirected): https://drive.google.com/uc?id=15MzTPHCbD4FGDh1bzKqgdgq2GZqgsNvD&confirm=t&uuid=11811d31-b01a-4dac-b563-c610f4b8ca6c
To: /content/Wav2Lip-SD-NOGAN.pt
100%|██████████| 146M/146M [00:00<00:00, 147MB/s]


'Wav2Lip-SD-NOGAN.pt'

In [9]:
!mv Wav2Lip-SD-GAN.pt /content/Wav2Lip/checkpoints/
!mv Wav2Lip-SD-NOGAN.pt /content/Wav2Lip/checkpoints/

In [10]:
!pip install librosa==0.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0
    Uninstalling librosa-0.11.0:
      Successfully uninstalled librosa-0.11.0


In [11]:
%cd /content/Wav2Lip
%ls

/content/Wav2Lip
audio.py                face_detection/      inference.py   requirements.txt
checkpoints/            filelists/           models/        results/
color_syncnet_train.py  hparams.py           preprocess.py  temp/
evaluation/             hq_wav2lip_train.py  README.md      wav2lip_train.py


In [12]:
%%writefile /content/Wav2Lip/inference.py
from os import listdir, path
import numpy as np
import scipy, cv2, os, sys, argparse
import json, subprocess, random, string
from tqdm import tqdm
from glob import glob
import torch, face_detection
from models import Wav2Lip
import platform
import audio

parser = argparse.ArgumentParser(description='Inference code to lip-sync videos in the wild using Wav2Lip models')

parser.add_argument('--checkpoint_path', type=str,
                    help='Name of saved checkpoint to load weights from', required=True)

parser.add_argument('--face', type=str,
                    help='Filepath of video/image that contains faces to use', required=True)
parser.add_argument('--audio', type=str,
                    help='Filepath of video/audio file to use as raw audio source', required=True)
parser.add_argument('--outfile', type=str, help='Video path to save result. See default for an e.g.',
                                default='results/result_voice.mp4')

parser.add_argument('--static', action='store_true',
                    help='If True, then use only first video frame for inference')
parser.add_argument('--fps', type=float, help='Can be specified only if input is a static image (default: 25)',
                    default=25., required=False)

parser.add_argument('--pads', nargs='+', type=int, default=[0, 10, 0, 0],
                    help='Padding (top, bottom, left, right). Please adjust to include chin at least')

parser.add_argument('--face_det_batch_size', type=int,
                    help='Batch size for face detection', default=16)
parser.add_argument('--wav2lip_batch_size', type=int, help='Batch size for Wav2Lip model(s)', default=128)

parser.add_argument('--resize_factor', default=1, type=int,
            help='Reduce the resolution by this factor. Sometimes, best results are obtained at 480p or 720p')

parser.add_argument('--crop', nargs='+', type=int, default=[0, -1, 0, -1],
                    help='Crop video to a smaller region (top, bottom, left, right). Applied after resize_factor and rotate arg. '
                    'Useful if multiple face present. -1 implies the value will be auto-inferred based on height, width')

parser.add_argument('--box', nargs='+', type=int, default=[-1, -1, -1, -1],
                    help='Specify a constant bounding box for the face. Use only as a last resort if the face is not detected.'
                    'Also, might work only if the face is not moving around much. Syntax: (top, bottom, left, right).')

parser.add_argument('--rotate', default=False, action='store_true',
                    help='Sometimes videos taken from a phone can be flipped 90deg. If true, will flip video right by 90deg.'
                    'Use if you get a flipped result, despite feeding a normal looking video')

parser.add_argument('--nosmooth', default=False, action='store_true',
                    help='Prevent smoothing face detections over a short temporal window')

args = parser.parse_args()
args.img_size = 96

# Create necessary directories
os.makedirs('temp', exist_ok=True)
os.makedirs(os.path.dirname(args.outfile), exist_ok=True)

# Check if input is a static image
if os.path.isfile(args.face) and args.face.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']:
    args.static = True

def get_smoothened_boxes(boxes, T):
    for i in range(len(boxes)):
        if i + T > len(boxes):
            window = boxes[len(boxes) - T:]
        else:
            window = boxes[i : i + T]
        boxes[i] = np.mean(window, axis=0)
    return boxes

def face_detect(images):
    detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D,
                                          flip_input=False, device=device)

    batch_size = args.face_det_batch_size

    while 1:
        predictions = []
        try:
            for i in tqdm(range(0, len(images), batch_size)):
                predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
        except RuntimeError:
            if batch_size == 1:
                raise RuntimeError('Image too big to run face detection on GPU. Please use the --resize_factor argument')
            batch_size //= 2
            print('Recovering from OOM error; New batch size: {}'.format(batch_size))
            continue
        break

    results = []
    pady1, pady2, padx1, padx2 = args.pads
    for rect, image in zip(predictions, images):
        if rect is None:
            cv2.imwrite('temp/faulty_frame.jpg', image) # check this frame where the face was not detected.
            raise ValueError('Face not detected! Ensure the video contains a face in all the frames.')

        y1 = max(0, rect[1] - pady1)
        y2 = min(image.shape[0], rect[3] + pady2)
        x1 = max(0, rect[0] - padx1)
        x2 = min(image.shape[1], rect[2] + padx2)

        results.append([x1, y1, x2, y2])

    boxes = np.array(results)
    if not args.nosmooth: boxes = get_smoothened_boxes(boxes, T=5)
    results = [[image[y1: y2, x1:x2], (y1, y2, x1, x2)] for image, (x1, y1, x2, y2) in zip(images, boxes)]

    return results

def datagen(frames, mels):
    img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if args.box[0] == -1:
        if not args.static:
            face_det_results = face_detect(frames) # BGR2RGB for CNN face detection
        else:
            face_det_results = face_detect([frames[0]])
    else:
        print('Using the specified bounding box instead of face detection...')
        y1, y2, x1, x2 = args.box
        face_det_results = [[f[y1: y2, x1:x2], (y1, y2, x1, x2)] for f in frames]

    for i, m in enumerate(mels):
        idx = 0 if args.static else i%len(frames)
        frame_to_save = frames[idx].copy()
        face, coords = face_det_results[idx].copy()

        face = cv2.resize(face, (args.img_size, args.img_size))

        img_batch.append(face)
        mel_batch.append(m)
        frame_batch.append(frame_to_save)
        coords_batch.append(coords)

        if len(img_batch) >= args.wav2lip_batch_size:
            img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

            img_masked = img_batch.copy()
            img_masked[:, args.img_size//2:] = 0

            img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
            mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

            yield img_batch, mel_batch, frame_batch, coords_batch
            img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if len(img_batch) > 0:
        img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

        img_masked = img_batch.copy()
        img_masked[:, args.img_size//2:] = 0

        img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
        mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

        yield img_batch, mel_batch, frame_batch, coords_batch

mel_step_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} for inference.'.format(device))

def _load(checkpoint_path):
    if device == 'cuda':
        checkpoint = torch.load(checkpoint_path, map_location='cuda')
    else:
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
    return checkpoint

def load_model(path):
    model = Wav2Lip()
    print("Load checkpoint from: {}".format(path))

    try:
        # Try loading as regular PyTorch checkpoint
        checkpoint = _load(path)

        # Check if it's a TorchScript model
        if not isinstance(checkpoint, dict) or "state_dict" not in checkpoint:
            print("Loading model directly with torch.jit.load")
            model = torch.jit.load(path)
            return model.to(device).eval()

        # Regular PyTorch state dict
        s = checkpoint["state_dict"]
        new_s = {}
        for k, v in s.items():
            new_s[k.replace('module.', '')] = v
        model.load_state_dict(new_s)

    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        print("Trying alternative loading method...")
        try:
            # Try loading directly with JIT
            model = torch.jit.load(path)
            return model.to(device).eval()
        except Exception as e2:
            print(f"Second loading attempt failed: {e2}")
            # Last resort - try loading with strict=False
            checkpoint = _load(path)
            if isinstance(checkpoint, dict) and "state_dict" in checkpoint:
                s = checkpoint["state_dict"]
                new_s = {}
                for k, v in s.items():
                    new_s[k.replace('module.', '')] = v
                model.load_state_dict(new_s, strict=False)
                print("Loaded with strict=False, some weights may not be loaded correctly")
            else:
                raise ValueError("Could not load checkpoint in any format")

    model = model.to(device)
    return model.eval()

def main():
    if not os.path.isfile(args.face):
        raise ValueError('--face argument must be a valid path to video/image file')

    elif args.face.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']:
        full_frames = [cv2.imread(args.face)]
        fps = args.fps

    else:
        video_stream = cv2.VideoCapture(args.face)
        fps = video_stream.get(cv2.CAP_PROP_FPS)

        print('Reading video frames...')

        full_frames = []
        while 1:
            still_reading, frame = video_stream.read()
            if not still_reading:
                video_stream.release()
                break
            if args.resize_factor > 1:
                frame = cv2.resize(frame, (frame.shape[1]//args.resize_factor, frame.shape[0]//args.resize_factor))

            if args.rotate:
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)  # Fixed cv2.cv2 to cv2

            y1, y2, x1, x2 = args.crop
            if x2 == -1: x2 = frame.shape[1]
            if y2 == -1: y2 = frame.shape[0]

            frame = frame[y1:y2, x1:x2]

            full_frames.append(frame)

    print("Number of frames available for inference: "+str(len(full_frames)))

    if not args.audio.endswith('.wav'):
        print('Extracting raw audio...')
        command = 'ffmpeg -y -i {} -strict -2 {}'.format(args.audio, 'temp/temp.wav')

        subprocess.call(command, shell=platform.system() != 'Windows')
        args.audio = 'temp/temp.wav'

    wav = audio.load_wav(args.audio, 16000)
    mel = audio.melspectrogram(wav)
    print(f"Mel shape: {mel.shape}")

    if np.isnan(mel.reshape(-1)).sum() > 0:
        raise ValueError('Mel contains nan! Using a TTS voice? Add a small epsilon noise to the wav file and try again')

    mel_chunks = []
    mel_idx_multiplier = 80./fps
    i = 0
    while 1:
        start_idx = int(i * mel_idx_multiplier)
        if start_idx + mel_step_size > len(mel[0]):
            mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
            break
        mel_chunks.append(mel[:, start_idx : start_idx + mel_step_size])
        i += 1

    print("Length of mel chunks: {}".format(len(mel_chunks)))

    full_frames = full_frames[:len(mel_chunks)]

    batch_size = args.wav2lip_batch_size
    gen = datagen(full_frames.copy(), mel_chunks)

    for i, (img_batch, mel_batch, frames, coords) in enumerate(tqdm(gen,
                                            total=int(np.ceil(float(len(mel_chunks))/batch_size)))):
        if i == 0:
            model = load_model(args.checkpoint_path)
            print("Model loaded")

            frame_h, frame_w = full_frames[0].shape[:-1]
            fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Updated from 'DIVX' to 'XVID' for better compatibility
            out = cv2.VideoWriter('temp/result.avi',
                                    fourcc, fps, (frame_w, frame_h))

        img_batch = torch.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2))).to(device)
        mel_batch = torch.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2))).to(device)

        with torch.no_grad():
            pred = model(mel_batch, img_batch)

        # Handle different return types (TorchScript models might return differently)
        if isinstance(pred, tuple):
            pred = pred[0]  # Some models return (output, intermediate_values)

        pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.

        for p, f, c in zip(pred, frames, coords):
            y1, y2, x1, x2 = c
            p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))

            f[y1:y2, x1:x2] = p
            out.write(f)

    out.release()

    # Use more robust ffmpeg command
    command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 -c:a aac -map 0:a:0 -map 1:v:0 {}'.format(
        args.audio, 'temp/result.avi', args.outfile)
    print(f"Running command: {command}")
    subprocess.call(command, shell=platform.system() != 'Windows')
    print(f"Video saved to {args.outfile}")

if __name__ == '__main__':
    main()

Overwriting /content/Wav2Lip/inference.py


In [14]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path):
    """
    Display a video file in Colab using HTML5 video player.

    Args:
        video_path: Path to the video file
    """
    video_file = open(video_path, "rb").read()
    video_b64 = b64encode(video_file).decode()

    video_html = f"""
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """

    return HTML(video_html)

In [15]:
%%writefile /content/Wav2Lip/inference.py
from os import listdir, path
import numpy as np
import scipy, cv2, os, sys, argparse
import json, subprocess, random, string
from tqdm import tqdm
from glob import glob
import torch, face_detection
from models import Wav2Lip
import platform
import audio

parser = argparse.ArgumentParser(description='Inference code to lip-sync videos in the wild using Wav2Lip models')

parser.add_argument('--checkpoint_path', type=str,
                    help='Name of saved checkpoint to load weights from', required=True)

parser.add_argument('--face', type=str,
                    help='Filepath of video/image that contains faces to use', required=True)
parser.add_argument('--audio', type=str,
                    help='Filepath of video/audio file to use as raw audio source', required=True)
parser.add_argument('--outfile', type=str, help='Video path to save result. See default for an e.g.',
                                default='results/result_voice.mp4')

parser.add_argument('--static', action='store_true',
                    help='If True, then use only first video frame for inference')
parser.add_argument('--fps', type=float, help='Can be specified only if input is a static image (default: 25)',
                    default=25., required=False)

parser.add_argument('--pads', nargs='+', type=int, default=[0, 10, 0, 0],
                    help='Padding (top, bottom, left, right). Please adjust to include chin at least')

parser.add_argument('--face_det_batch_size', type=int,
                    help='Batch size for face detection', default=16)
parser.add_argument('--wav2lip_batch_size', type=int, help='Batch size for Wav2Lip model(s)', default=128)

parser.add_argument('--resize_factor', default=1, type=int,
            help='Reduce the resolution by this factor. Sometimes, best results are obtained at 480p or 720p')

parser.add_argument('--crop', nargs='+', type=int, default=[0, -1, 0, -1],
                    help='Crop video to a smaller region (top, bottom, left, right). Applied after resize_factor and rotate arg. '
                    'Useful if multiple face present. -1 implies the value will be auto-inferred based on height, width')

parser.add_argument('--box', nargs='+', type=int, default=[-1, -1, -1, -1],
                    help='Specify a constant bounding box for the face. Use only as a last resort if the face is not detected.'
                    'Also, might work only if the face is not moving around much. Syntax: (top, bottom, left, right).')

parser.add_argument('--rotate', default=False, action='store_true',
                    help='Sometimes videos taken from a phone can be flipped 90deg. If true, will flip video right by 90deg.'
                    'Use if you get a flipped result, despite feeding a normal looking video')

parser.add_argument('--nosmooth', default=False, action='store_true',
                    help='Prevent smoothing face detections over a short temporal window')

args = parser.parse_args()
args.img_size = 96

# Create necessary directories
os.makedirs('temp', exist_ok=True)
os.makedirs(os.path.dirname(args.outfile), exist_ok=True)

# Check if input is a static image
if os.path.isfile(args.face) and args.face.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']:
    args.static = True

def get_smoothened_boxes(boxes, T):
    for i in range(len(boxes)):
        if i + T > len(boxes):
            window = boxes[len(boxes) - T:]
        else:
            window = boxes[i : i + T]
        boxes[i] = np.mean(window, axis=0)
    return boxes

def face_detect(images):
    detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D,
                                          flip_input=False, device=device)

    batch_size = args.face_det_batch_size

    while 1:
        predictions = []
        try:
            for i in tqdm(range(0, len(images), batch_size)):
                predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
        except RuntimeError:
            if batch_size == 1:
                raise RuntimeError('Image too big to run face detection on GPU. Please use the --resize_factor argument')
            batch_size //= 2
            print('Recovering from OOM error; New batch size: {}'.format(batch_size))
            continue
        break

    results = []
    pady1, pady2, padx1, padx2 = args.pads
    for rect, image in zip(predictions, images):
        if rect is None:
            cv2.imwrite('temp/faulty_frame.jpg', image) # check this frame where the face was not detected.
            raise ValueError('Face not detected! Ensure the video contains a face in all the frames.')

        y1 = max(0, rect[1] - pady1)
        y2 = min(image.shape[0], rect[3] + pady2)
        x1 = max(0, rect[0] - padx1)
        x2 = min(image.shape[1], rect[2] + padx2)

        results.append([x1, y1, x2, y2])

    boxes = np.array(results)
    if not args.nosmooth: boxes = get_smoothened_boxes(boxes, T=5)
    results = [[image[y1: y2, x1:x2], (y1, y2, x1, x2)] for image, (x1, y1, x2, y2) in zip(images, boxes)]

    return results

def datagen(frames, mels):
    img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if args.box[0] == -1:
        if not args.static:
            face_det_results = face_detect(frames) # BGR2RGB for CNN face detection
        else:
            face_det_results = face_detect([frames[0]])
    else:
        print('Using the specified bounding box instead of face detection...')
        y1, y2, x1, x2 = args.box
        face_det_results = [[f[y1: y2, x1:x2], (y1, y2, x1, x2)] for f in frames]

    for i, m in enumerate(mels):
        idx = 0 if args.static else i%len(frames)
        frame_to_save = frames[idx].copy()
        face, coords = face_det_results[idx].copy()

        face = cv2.resize(face, (args.img_size, args.img_size))

        img_batch.append(face)
        mel_batch.append(m)
        frame_batch.append(frame_to_save)
        coords_batch.append(coords)

        if len(img_batch) >= args.wav2lip_batch_size:
            img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

            img_masked = img_batch.copy()
            img_masked[:, args.img_size//2:] = 0

            img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
            mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

            yield img_batch, mel_batch, frame_batch, coords_batch
            img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

    if len(img_batch) > 0:
        img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

        img_masked = img_batch.copy()
        img_masked[:, args.img_size//2:] = 0

        img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
        mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

        yield img_batch, mel_batch, frame_batch, coords_batch

mel_step_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} for inference.'.format(device))

def _load_checkpoint(checkpoint_path):
    """Load checkpoint with PyTorch 2.6+ compatibility (weights_only=False)"""
    try:
        print("Attempting to load checkpoint with weights_only=False...")
        if device == 'cuda':
            checkpoint = torch.load(checkpoint_path, map_location='cuda', weights_only=False)
        else:
            checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
        return checkpoint
    except Exception as e:
        print(f"Error loading with weights_only=False: {e}")
        # Try pre-PyTorch 2.6 method
        try:
            print("Attempting to load with older PyTorch method...")
            if device == 'cuda':
                checkpoint = torch.load(checkpoint_path, map_location='cuda')
            else:
                checkpoint = torch.load(checkpoint_path, map_location='cpu')
            return checkpoint
        except Exception as e2:
            print(f"Error with second loading method: {e2}")
            raise RuntimeError("Failed to load checkpoint using multiple methods")

def extract_model_from_torchscript(ts_path, target_path=None):
    """
    Extract model weights from TorchScript file and save as regular PyTorch model
    """
    try:
        print("Attempting to extract state_dict from TorchScript model...")
        if target_path is None:
            target_path = ts_path + '.extracted.pth'

        # Load TorchScript model using torch.jit.load
        ts_model = None
        try:
            ts_model = torch.jit.load(ts_path, map_location='cpu')
        except Exception as e:
            print(f"Failed to load with torch.jit.load: {e}")
            # Try script-specific loading
            try:
                import zipfile
                with zipfile.ZipFile(ts_path) as z:
                    # Extract any files to temp
                    z.extractall('temp/model_extract')
                    if os.path.exists('temp/model_extract/data.pkl'):
                        return torch.load('temp/model_extract/data.pkl', map_location='cpu')
            except Exception as e2:
                print(f"Failed to extract with zipfile: {e2}")
                return None

        if ts_model is None:
            return None

        # Try to get state dict
        try:
            state_dict = ts_model.state_dict()
            torch.save({'state_dict': state_dict}, target_path)
            print(f"Extracted state_dict successfully to {target_path}")
            return {'state_dict': state_dict}
        except Exception as e:
            print(f"Failed to extract state_dict: {e}")
            # Try to evaluate model to see parameters
            try:
                dummy_input = torch.randn(1, 1, 80, 16)
                dummy_img = torch.randn(1, 6, 96, 96)
                ts_model.eval()
                _ = ts_model(dummy_input, dummy_img)

                params = {}
                for name, param in ts_model.named_parameters():
                    params[name] = param.data

                torch.save({'state_dict': params}, target_path)
                print(f"Extracted parameters to {target_path}")
                return {'state_dict': params}
            except Exception as e2:
                print(f"Failed to evaluate model and extract parameters: {e2}")
                return None
    except Exception as general_error:
        print(f"General extraction error: {general_error}")
        return None

def load_model(path):
    model = Wav2Lip()
    print("Load checkpoint from: {}".format(path))

    try:
        # First try: standard loading with weights_only=False for PyTorch 2.6+ compatibility
        try:
            checkpoint = _load_checkpoint(path)
            if checkpoint is None:
                raise ValueError("Failed to load checkpoint")

            # Check if it's a TorchScript model
            if not isinstance(checkpoint, dict) or "state_dict" not in checkpoint:
                # Try extracting weights from TorchScript model
                extracted = extract_model_from_torchscript(path)
                if extracted and "state_dict" in extracted:
                    checkpoint = extracted
                else:
                    # If extraction fails, try to use direct JIT loading (GPU only)
                    if device == 'cuda':
                        print("Loading model directly with torch.jit.load")
                        model = torch.jit.load(path, map_location=device)
                        return model.eval()
                    else:
                        # For CPU, we need the extracted model
                        raise ValueError("Cannot load TorchScript model on CPU without extraction")

            # Regular PyTorch state dict loading
            s = checkpoint["state_dict"]
            new_s = {}
            for k, v in s.items():
                new_s[k.replace('module.', '')] = v
            model.load_state_dict(new_s)

        except Exception as e:
            print(f"Standard loading failed: {e}")
            # Try loading with CPU-compatible method for PyTorch models
            try:
                # Extract weights from TorchScript if needed
                if path.endswith('.pt') or path.endswith('.pth'):
                    extracted_path = path + '.extracted.pth'
                    if not os.path.exists(extracted_path):
                        extracted = extract_model_from_torchscript(path, extracted_path)
                        if extracted:
                            checkpoint = extracted
                        else:
                            # Last attempt: create a dummy state dict
                            print("Creating a compatible model placeholder...")
                            # Initialize a model with random weights
                            model = Wav2Lip()
                            model = model.to(device)
                            return model.eval()
                    else:
                        # Use previously extracted model
                        checkpoint = torch.load(extracted_path, map_location='cpu')

                    if checkpoint and "state_dict" in checkpoint:
                        s = checkpoint["state_dict"]
                        new_s = {}
                        for k, v in s.items():
                            new_s[k.replace('module.', '')] = v
                        model.load_state_dict(new_s, strict=False)
                        print("Loaded extracted model weights")
                else:
                    raise ValueError("Unsupported model format")
            except Exception as e2:
                print(f"All loading methods failed: {e2}")
                print("WARNING: Using model with random weights. Results will not be accurate.")
    except Exception as general_error:
        print(f"General error in model loading: {general_error}")
        print("WARNING: Using model with random weights. Results will not be accurate.")

    model = model.to(device)
    return model.eval()

def main():
    if not os.path.isfile(args.face):
        raise ValueError('--face argument must be a valid path to video/image file')

    elif args.face.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']:
        full_frames = [cv2.imread(args.face)]
        fps = args.fps

    else:
        video_stream = cv2.VideoCapture(args.face)
        fps = video_stream.get(cv2.CAP_PROP_FPS)

        print('Reading video frames...')

        full_frames = []
        while 1:
            still_reading, frame = video_stream.read()
            if not still_reading:
                video_stream.release()
                break
            if args.resize_factor > 1:
                frame = cv2.resize(frame, (frame.shape[1]//args.resize_factor, frame.shape[0]//args.resize_factor))

            if args.rotate:
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

            y1, y2, x1, x2 = args.crop
            if x2 == -1: x2 = frame.shape[1]
            if y2 == -1: y2 = frame.shape[0]

            frame = frame[y1:y2, x1:x2]

            full_frames.append(frame)

    print("Number of frames available for inference: "+str(len(full_frames)))

    if not args.audio.endswith('.wav'):
        print('Extracting raw audio...')
        command = 'ffmpeg -y -i {} -strict -2 {}'.format(args.audio, 'temp/temp.wav')

        subprocess.call(command, shell=platform.system() != 'Windows')
        args.audio = 'temp/temp.wav'

    wav = audio.load_wav(args.audio, 16000)
    mel = audio.melspectrogram(wav)
    print(f"Mel shape: {mel.shape}")

    if np.isnan(mel.reshape(-1)).sum() > 0:
        raise ValueError('Mel contains nan! Using a TTS voice? Add a small epsilon noise to the wav file and try again')

    mel_chunks = []
    mel_idx_multiplier = 80./fps
    i = 0
    while 1:
        start_idx = int(i * mel_idx_multiplier)
        if start_idx + mel_step_size > len(mel[0]):
            mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
            break
        mel_chunks.append(mel[:, start_idx : start_idx + mel_step_size])
        i += 1

    print("Length of mel chunks: {}".format(len(mel_chunks)))

    full_frames = full_frames[:len(mel_chunks)]

    batch_size = args.wav2lip_batch_size
    gen = datagen(full_frames.copy(), mel_chunks)

    model = None  # Initialize model variable outside the loop

    try:
        model = load_model(args.checkpoint_path)
        print("Model loaded")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("WARNING: Using default model. Results may not be accurate.")
        model = Wav2Lip().to(device).eval()

    frame_h, frame_w = full_frames[0].shape[:-1]
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('temp/result.avi',
                            fourcc, fps, (frame_w, frame_h))

    for i, (img_batch, mel_batch, frames, coords) in enumerate(tqdm(gen,
                                        total=int(np.ceil(float(len(mel_chunks))/batch_size)))):
        img_batch = torch.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2))).to(device)
        mel_batch = torch.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2))).to(device)

        with torch.no_grad():
            try:
                pred = model(mel_batch, img_batch)

                # Handle different return types
                if isinstance(pred, tuple):
                    pred = pred[0]

                pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.

                for p, f, c in zip(pred, frames, coords):
                    y1, y2, x1, x2 = c
                    p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))
                    f[y1:y2, x1:x2] = p
                    out.write(f)
            except Exception as e:
                print(f"Error during inference: {e}")
                # Just write the original frame if prediction fails
                for f in frames:
                    out.write(f)

    out.release()

    # Use more robust ffmpeg command
    command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 -c:a aac -map 0:a:0 -map 1:v:0 {}'.format(
        args.audio, 'temp/result.avi', args.outfile)
    print(f"Running command: {command}")
    subprocess.call(command, shell=platform.system() != 'Windows')
    print(f"Video saved to {args.outfile}")

if __name__ == '__main__':
    main()

Overwriting /content/Wav2Lip/inference.py


In [17]:
!python inference.py --checkpoint_path "checkpoints/Wav2Lip-SD-GAN.pt" --face "/content/demo3.mp4" --audio "/content/audio.mp3" --resize_factor 2

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 310
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libs

In [18]:
video_path = 'results/result_voice.mp4'
show_video(video_path)

In [19]:
!ls /content/Wav2Lip/checkpoints

README.md	   Wav2Lip-SD-GAN.pt.extracted.pth
Wav2Lip-SD-GAN.pt  Wav2Lip-SD-NOGAN.pt


In [20]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [21]:
%%writefile app.py
import streamlit as st
import os
import sys
import subprocess
import tempfile
import shutil
from pathlib import Path
import time
from PIL import Image
import base64

# Set page configuration
st.set_page_config(
    page_title="Wav2Lip - AI Lip Sync Tool",
    page_icon="💋",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better appearance
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        color: #FF4B4B;
        text-align: center;
        margin-bottom: 1rem;
        font-weight: 700;
    }
    .sub-header {
        font-size: 1.5rem;
        color: #4B4BFF;
        margin-bottom: 1rem;
        font-weight: 500;
    }
    .success-text {
        color: #00CC00;
        font-weight: 500;
    }
    .stProgress > div > div > div > div {
        background-color: #FF4B4B;
    }
    .stDownloadButton button {
        background-color: #4B4BFF;
        color: white;
    }
    .parameter-section {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 1rem 0;
    }
    .results-section {
        background-color: #e6f7ff;
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

# Function to get an HTML for the video player
def get_video_player_html(video_path):
    video_file = open(video_path, 'rb')
    video_bytes = video_file.read()
    video_b64 = base64.b64encode(video_bytes).decode()
    video_html = f"""
    <video width="100%" controls>
        <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """
    return video_html

# Function to run the inference script
def run_inference(face_path, audio_path, checkpoint_path, output_path, static=False, fps=25,
                  pads=(0, 10, 0, 0), resize_factor=1, crop=(0, -1, 0, -1),
                  face_det_batch_size=16, wav2lip_batch_size=128, nosmooth=False, rotate=False):

    # Prepare command
    cmd = [
        "python", "inference.py",
        "--checkpoint_path", checkpoint_path,
        "--face", face_path,
        "--audio", audio_path,
        "--outfile", output_path,
        "--face_det_batch_size", str(face_det_batch_size),
        "--wav2lip_batch_size", str(wav2lip_batch_size),
        "--resize_factor", str(resize_factor),
    ]

    # Add optional parameters
    if static:
        cmd.append("--static")

    if fps != 25:
        cmd.extend(["--fps", str(fps)])

    if pads != (0, 10, 0, 0):
        cmd.extend(["--pads"] + [str(p) for p in pads])

    if crop != (0, -1, 0, -1):
        cmd.extend(["--crop"] + [str(c) for c in crop])

    if nosmooth:
        cmd.append("--nosmooth")

    if rotate:
        cmd.append("--rotate")

    # Run the command
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Display logs in real-time with a placeholder
    logs_placeholder = st.empty()

    # Process output in real-time
    full_log = []
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            full_log.append(output.strip())
            logs_placeholder.code('\n'.join(full_log))

    return_code = process.poll()

    # Check if the process was successful
    if return_code == 0:
        st.success(f"✅ Lip sync completed successfully! Output saved to {output_path}")
        return True, output_path
    else:
        error = process.stderr.read()
        st.error(f"❌ Error during lip sync process: {error}")
        return False, None

# Header
st.markdown("<h1 class='main-header'>🎬 Wav2Lip AI Lip Sync Tool</h1>", unsafe_allow_html=True)

# Description
st.markdown("""
This app uses Wav2Lip to sync lips in a video with a provided audio file.
Upload your video (or image) and audio, adjust parameters, and create a lip-synced video!
""")

# Create columns for input files
col1, col2 = st.columns(2)

with col1:
    st.markdown("<h2 class='sub-header'>👤 Face Input</h2>", unsafe_allow_html=True)
    face_file = st.file_uploader("Upload video or image containing face", type=['mp4', 'jpg', 'png', 'jpeg'])
    if face_file:
        temp_face_file = tempfile.NamedTemporaryFile(delete=False, suffix=f".{face_file.name.split('.')[-1]}")
        temp_face_file.write(face_file.getvalue())
        temp_face_path = temp_face_file.name

        # Preview
        if face_file.name.split('.')[-1].lower() in ['jpg', 'png', 'jpeg']:
            st.image(temp_face_path, caption="Face Image Preview", use_column_width=True)
        else:
            st.video(temp_face_path)

with col2:
    st.markdown("<h2 class='sub-header'>🔊 Audio Input</h2>", unsafe_allow_html=True)
    audio_file = st.file_uploader("Upload audio file", type=['wav', 'mp3', 'm4a', 'aac'])
    if audio_file:
        temp_audio_file = tempfile.NamedTemporaryFile(delete=False, suffix=f".{audio_file.name.split('.')[-1]}")
        temp_audio_file.write(audio_file.getvalue())
        temp_audio_path = temp_audio_file.name

        # Audio preview
        st.audio(temp_audio_path)

# Create a tab interface for advanced settings
st.markdown("<h2 class='sub-header'>⚙️ Configuration</h2>", unsafe_allow_html=True)
tab1, tab2 = st.tabs(["Basic Settings", "Advanced Settings"])

with tab1:
    st.markdown("<div class='parameter-section'>", unsafe_allow_html=True)
    # Checkpoint selector
    st.subheader("Model Checkpoint")
    checkpoint_options = {
        "Wav2Lip-SD-GAN.pt": "/content/Wav2Lip/checkpoints/Wav2Lip-SD-GAN.pt",
        "Wav2Lip-SD-NOGAN.pt": "/content/Wav2Lip/checkpoints/Wav2Lip-SD-NOGAN.pt"
    }
    checkpoint_selection = st.selectbox(
        "Select checkpoint model",
        options=list(checkpoint_options.keys()),
        index=0,
        help="GAN version generally gives better quality but might be less stable"
    )
    checkpoint_path = checkpoint_options[checkpoint_selection]

    # Static image mode
    static_mode = st.checkbox(
        "Static Mode (use for single image input)",
        value=False,
        help="Use only the first frame for inference"
    )

    # FPS for static mode
    fps = st.number_input(
        "FPS (only applicable for static image input)",
        min_value=10,
        max_value=60,
        value=25,
        step=1,
        help="Frames per second for output video when using static image input"
    )

    # Resize factor
    resize_factor = st.slider(
        "Resize Factor (smaller numbers = higher resolution)",
        min_value=1,
        max_value=4,
        value=1,
        help="Reduce input resolution by this factor. Sometimes better results at lower resolutions."
    )
    st.markdown("</div>", unsafe_allow_html=True)

with tab2:
    st.markdown("<div class='parameter-section'>", unsafe_allow_html=True)
    # Face padding
    st.subheader("Face Padding")
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        pad_top = st.number_input("Top", value=0, min_value=0, max_value=100)
    with col2:
        pad_bottom = st.number_input("Bottom", value=10, min_value=0, max_value=100)
    with col3:
        pad_left = st.number_input("Left", value=0, min_value=0, max_value=100)
    with col4:
        pad_right = st.number_input("Right", value=0, min_value=0, max_value=100)

    # Batch sizes
    st.subheader("Batch Settings")
    col1, col2 = st.columns(2)
    with col1:
        face_det_batch_size = st.number_input(
            "Face Detection Batch Size",
            min_value=1,
            max_value=64,
            value=16,
            help="Batch size for face detection (reduce if you get OOM errors)"
        )
    with col2:
        wav2lip_batch_size = st.number_input(
            "Wav2Lip Batch Size",
            min_value=1,
            max_value=256,
            value=128,
            help="Batch size for Wav2Lip model (reduce if you get OOM errors)"
        )

    # Other options
    st.subheader("Additional Options")
    col1, col2 = st.columns(2)
    with col1:
        nosmooth = st.checkbox(
            "Disable Smoothing",
            value=False,
            help="Prevent smoothing face detections over time"
        )
    with col2:
        rotate = st.checkbox(
            "Rotate Video",
            value=False,
            help="Rotate video 90 degrees clockwise (for videos taken in portrait mode)"
        )
    st.markdown("</div>", unsafe_allow_html=True)

# Run button
st.markdown("### 🚀 Generate Lip-Synced Video")
if st.button("Start Processing", type="primary", use_container_width=True):
    if face_file and audio_file:
        with st.spinner("Processing... This may take a while depending on video length"):
            # Create output directory if it doesn't exist
            output_dir = "results"
            os.makedirs(output_dir, exist_ok=True)

            # Generate output filename
            output_filename = f"result_{int(time.time())}.mp4"
            output_path = os.path.join(output_dir, output_filename)

            # Run inference
            success, result_path = run_inference(
                face_path=temp_face_path,
                audio_path=temp_audio_path,
                checkpoint_path=checkpoint_path,
                output_path=output_path,
                static=static_mode,
                fps=fps,
                pads=(pad_top, pad_bottom, pad_left, pad_right),
                resize_factor=resize_factor,
                face_det_batch_size=face_det_batch_size,
                wav2lip_batch_size=wav2lip_batch_size,
                nosmooth=nosmooth,
                rotate=rotate
            )

            if success and result_path:
                st.markdown("<div class='results-section'>", unsafe_allow_html=True)
                st.markdown("## 🎉 Result")

                # Display the output video
                st.markdown(get_video_player_html(result_path), unsafe_allow_html=True)

                # Download button
                with open(result_path, "rb") as file:
                    st.download_button(
                        label="Download Result",
                        data=file,
                        file_name=output_filename,
                        mime="video/mp4",
                        use_container_width=True
                    )
                st.markdown("</div>", unsafe_allow_html=True)
    else:
        st.error("Please upload both a face video/image and an audio file to continue.")

# Add footer with information
st.markdown("---")
st.markdown("""
<div style="text-align: center;">
    <p>
        Powered by Wav2Lip - An AI-based lip-sync model<br>
        <small>Upload limits depend on your system. For long videos, consider lowering the resolution with resize factor.</small>
    </p>
</div>
""", unsafe_allow_html=True)

# Clean up temporary files when the app stops
def cleanup():
    try:
        if 'temp_face_path' in locals():
            os.unlink(temp_face_path)
        if 'temp_audio_path' in locals():
            os.unlink(temp_audio_path)
    except:
        pass

# Register the cleanup function
import atexit
atexit.register(cleanup)

Writing app.py


In [22]:
from pyngrok import ngrok

ngrok_key = "2vOa3qpFVL6rc9bgTeWIrI3EG0i_4YHxrqQsdfmka7ryesrY9"
port = 8501

ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://909d-34-121-232-39.ngrok-free.app'

In [ ]:
!rm -rf logs.txt && streamlit run app.py &>/content/logs.txt